# Import ELK Snapshots onto a Fabric slice

### Prerequisite
- A running fabric slice (For example: [Create single node backup Slice](./create_single_node_backup.ipynb))
- An ELK snapshot tar file to import

#### General Imports

In [2]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
except Exception as e:
    print(f"Exception: {e}")

#### Get your slice information
*Set slice_name and node_name below before running*

In [3]:
slice_name='backup-testing'
node_name='data-backup'


slice = fablib.get_slice(name=slice_name)
slice.show();
node = slice.get_node(name=node_name)
node.show();

ID,1858a90f-7dd1-4616-95f6-7752a5272da3
Name,backup-testing
Lease Expiration (UTC),2023-07-25 00:04:09 +0000
Lease Start (UTC),2023-07-17 23:59:34 +0000
Project ID,6b76128d-c73f-431f-a245-0397586a7d40
State,StableOK


ID,45245312-911c-47ad-8002-99a8a3356173
Name,data-backup
Cores,2
RAM,8
Disk,100
Image,docker_ubuntu_20
Image Type,qcow2
Host,cern-w5.fabric-testbed.net
Site,CERN
Username,ubuntu
Management IP,2001:400:a100:3090:f816:3eff:fe14:5ed8


#### Import ELK import/export service tools

In [4]:
from mflib.data_transfer import elk_service
elk_tools = elk_import_service(slice_name=slice_name, node_name=node_name)

# Instructions to start ELK and import your snapshot data

#### Ensure Docker and Docker-Compose are installed on the node

In [9]:
elk_tools.install_docker()

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Fetched 336 kB in 1s (312 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
docker is already the newest version (1.5-2).
0 upgraded, 0 newly installed, 0 to remove and 126 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
docker-compose is already the newest version (1.25.0-1).
0 upgraded, 0 newly installed, 0 to remove and 126 not upgraded.
Docker version 23.0.4, build f480fb1
docker-compose version 1.25.0, build unknown


#### Set up NAT64 Server
We need to do this step so that we can clone the Github repo with our docker app.

In [10]:
elk_tools.setup_nat64(node_name)

### Create docker container and import your data

#### Clone Docker App Repository
Clones https://github.com/jackhancockuky/elk-docker-container.git to your docker container.
This repository contains a docker-compose app which will allow you to automatically set up Elastic Search and Kibana on your node.

In [ ]:
elk_tools.clone_repository()

#### Upload a snapshot file into ELK

In [4]:
# Enter your elk snapshot tar file name (must be in data_transfer_service/snapshots directory)
snapshot_tar_name = "elk_backup_2.tar"

# 1. Uploads TAR file to your node
# 2. Untars data and places inside ELK's shared docker volume
elk_tools.import_snapshot(snapshot_tar_name)

Creating imported_data directory..
Moving snapshot file..
Untarring snapshot data into shared docker volume..
_data/
_data/snap-nx1kMYKkQNqbsfCL5iRLMg.dat
_data/meta-nx1kMYKkQNqbsfCL5iRLMg.dat
_data/indices/
_data/indices/z2E3XWORRIyAbx-Dr3w8lw/
_data/indices/z2E3XWORRIyAbx-Dr3w8lw/0/
_data/indices/z2E3XWORRIyAbx-Dr3w8lw/0/snap-nx1kMYKkQNqbsfCL5iRLMg.dat
_data/indices/z2E3XWORRIyAbx-Dr3w8lw/0/snap-zLt1yM6tRmmMpnqxZMjj2A.dat
_data/indices/z2E3XWORRIyAbx-Dr3w8lw/0/index-qFWoddmvTICVl6YnJYEgWw
_data/indices/z2E3XWORRIyAbx-Dr3w8lw/meta-Dbibi4gBMQKlzNGxJMF-.dat
_data/indices/16fzAGeARxaEMOT80y3Xrw/
_data/indices/16fzAGeARxaEMOT80y3Xrw/0/
_data/indices/16fzAGeARxaEMOT80y3Xrw/0/snap-nx1kMYKkQNqbsfCL5iRLMg.dat
_data/indices/16fzAGeARxaEMOT80y3Xrw/0/index-NLZmJaDSSty1NyT-4Agqbg
_data/indices/16fzAGeARxaEMOT80y3Xrw/0/snap-zLt1yM6tRmmMpnqxZMjj2A.dat
_data/indices/16fzAGeARxaEMOT80y3Xrw/meta-DLibi4gBMQKlzNGxJMF9.dat
_data/indices/qIeeCZxXS8uiq4TicG7NOA/
_data/indices/qIeeCZxXS8uiq4TicG7NOA/0/
_dat

#### Start the docker compose app

In [5]:
elk_tools.start_docker()

7.13.0: Pulling from elasticsearch/elasticsearch
 Creating network "elk-docker-container_elastic" with driver "bridge"
Creating volume "elk-docker-container_es-data" with local driver
Pulling elasticsearch (docker.elastic.co/elasticsearch/elasticsearch:7.13.0)...
 Digest: sha256:c7efa0260b40ebe8332c38b113b628bc33f78ab33feb97494a56d1338dc88c12
Status: Downloaded newer image for docker.elastic.co/elasticsearch/elasticsearch:7.13.0
 Pulling kibana (docker.elastic.co/kibana/kibana:7.13.0)...
 7.13.0: Pulling from kibana/kibana
Digest: sha256:2075b0a1c4881bda1536cd58fc5b240d37871419c9487fd6300c85ae5062f27c
Status: Downloaded newer image for docker.elastic.co/kibana/kibana:7.13.0
 Creating elasticsearch ... 
  Creating kibana        ... 
 kibana        ... done

#### Register the repository

In [4]:
# Choose a name for the repository your snapshot will be imported into or keep as default
repository_name = "imported_repository"

elk_tools.register_repository(repository_name)

{"acknowledged":true}   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   129  100    21  100   108    253   1301 --:--:-- --:--:-- --:--:--  1554
 

#### View repository after registration

In [7]:
elk_tools.view_repository("imported_repository")

id              status start_epoch start_time end_epoch  end_time duration indices successful_shards failed_shards total_shards
snapshot_test  SUCCESS 1685969379  12:49:39   1685969380 12:49:40    405ms       9                 9             0            9
snapshot_test1 SUCCESS 1685969393  12:49:53   1685969393 12:49:53    201ms       9                 9             0            9
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   384  100   384    0     0  29538      0 --:--:-- --:--:-- --:--:-- 29538
 

#### Delete all indices and restore snapshot (replace snapshot name and change index list if you want to customize)

In [7]:
repository_name = "imported_repository"
snapshot_name = "snapshot_test"

elk_tools.restore_snapshot(repository_name, snapshot_name)

{"acknowledged":true}   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    21  100    21    0     0  10500      0 --:--:-- --:--:-- --:--:-- 10500
 {
  "accepted" : true
}
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    93  100    24  100    69     58    169 --:--:-- --:--:-- --:--:--   227
 

### Your data is now imported on the node.

----
### View the index list

In [8]:
elk_tools.view_indices()

health status index                               uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   metricbeat-7.13.2-2023.06.05-000001 o4Kk8cflSOO5YXN-Pub4pg   1   1      12690            0        1mb            1mb
yellow open   filebeat-7.13.2-2023.06.05-000001   qx-8oNhdSGi1jhcrt8EOZQ   1   1       8477            0      2.2mb          2.2mb
green  open   .kibana_task_manager_7.13.0_001     WWsLfflwRTerBgESpyC5KA   1   0         10           20    102.4kb        102.4kb
green  open   .apm-custom-link                    7uY_PoY1TfWpknbbeyC2-w   1   0          0            0       208b           208b
green  open   .kibana-event-log-7.13.0-000001     sIv63RT_SBeLQKfl0KseRA   1   0          1            0      5.6kb          5.6kb
green  open   .apm-agent-configuration            ofB2mABpSlOdjQqpOEYMJg   1   0          0            0       208b           208b
green  open   .kibana_7.13.0_001                  TzCiKxT6R5WhWmMEtTuB7Q   1   0   

----
### View data from Kibana

#### On your local terminal create a tunnel with:

'ssh -L 10030:localhost:5601 -F ssh_config -i slice_key ubuntu@{node_ip}'

#### Then go to http://localhost:10030
----

### To reuse slice with new data (if you are done then just delete the slice)
#### Take down container and remove data

In [ ]:
#elk_tools.stop_docker()
#elk_tools.remove_data()

#### Rerun steps starting from *Create Docker container and import your data*